**python command to extract wikipedia articles fron the dump file:**
python .\WikiExtractor.py --output out500json --no_templates --min_text_length 500 --bytes 1073741824 --json .\arwiki-20180120-pages-articles-multistream.xml.bz2

In [1]:
import json
import pandas as pd
import re
from pathlib import Path
#from fastprogress import master_bar, progress_bar
from tqdm import tqdm

In [37]:
punc_to_remove = "!?"
punc_to_separate = ".,"
re.sub(r'([{}])'.format(punc_to_separate), r' \1 ', 'ok.! but, why? :(')

'ok . ! but ,  why? :('

In [70]:
import string

#preparing punctuations list
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ«»'''
english_punctuations = string.punctuation
punc_to_remove = ''.join(set(arabic_punctuations + english_punctuations))
punc_to_keep = '''!؟،+.'''
punc_to_escape = '''[]-^'''
for p in punc_to_keep: punc_to_remove = punc_to_remove.replace(p, '')
for p in punc_to_escape: punc_to_remove = punc_to_remove.replace(p, '\\{}'.format(p))
print(punc_to_remove)

\]×*,~$»=&\^_%(«#;<|“{…\-'–¦`\}”:?÷"\[؛/@ـ)>


In [2]:
def pre_process(text):
    text = text.replace('\\n', ' ').replace('\n', ' ')
    text = text.replace('؛', '،')
    text = re.sub(r'\([^)]+\)', '', text)  # remove parentheses and everything in between
    text = re.sub(r'[a-zA-Z]', '', text)  # remove non-arabic characters
    text = re.sub(r'\d+(\.\d+)?', ' رقم ', text)  # replace numbers by special token
    for p in punc_to_remove: text = text.replace(p, '')  # remove punctuations
    
    text = re.sub(r'\s+', r' ', text)
    return text

def parse_one_json_article(json_articles_file, output_file):
    with open(json_articles_file, encoding='utf-8') as file: # open json file
        line = file.readline()
        line = line.strip() # remove leading and trailing whitespaces (and new lines)
        article = json.loads(line)['text']
        article = pre_process(article)
        with open(output_file, 'w', encoding='utf-8') as out:
            out.write(article)
    print('DONE :)')

def parse_json_articles(path):
    """ parses articles formatted in json & extracted by WikiExtractor.py (https://github.com/attardi/wikiextractor)
        returns list of articles as dictionary: {"id", "url", "title", "text"}
    """
    jsons = []
    for f in path.iterdir(): # loop through files in directory
        with open(path/f.name, encoding='utf-8') as file: # open json file
            for line in file.readlines(): # loop through file lines. Each line is an article formatted in json
                line = line.strip() # remove leading and trailing whitespaces (and new lines)
                jsons.append(line)
    # create a json string: list of all articles
    json_string = '[{}]'.format(', '.join(jsons))
    # convert json string to python list
    articles_list = json.loads(json_string)
    return articles_list

def preprocess_articles_and_write_to_file(articles, output_file):
    with open(output_file, 'w', encoding='utf-8') as out:
        for i in tqdm(range(len(articles))):
            article = articles[i]
            text = pre_process(article['text'])
            out.write(text + '\n')
    print('DONE :)')

In [72]:
pre_process('يعود الأصل اللغوي لكلمة ماء في اللغة العربية إلى الجذر: (م و هـ)، تحرّكت الواو وانفتح ما قبلها فقُلبت ألفاً، فصار: (ماه)، ثم أبدلت الهاء همزة فصار: (ماء)، وتجمع الكلمة على "مياه" و"أمواه".')

'يعود الأصل اللغوي لكلمة ماء في اللغة العربية إلى الجذر ، تحرّكت الواو وانفتح ما قبلها فقُلبت ألفاً، فصار ، ثم أبدلت الهاء همزة فصار ، وتجمع الكلمة على مياه وأمواه.'

In [23]:
re.sub(r'([.،])', r' \1 ', 'الماء الموجود على الأرض، وأغلب هذه الكمّيّة (حوالي 99%) موجودة. يلا')

'الماء الموجود على الأرض ،  وأغلب هذه الكمّيّة (حوالي 99%) موجودة .  يلا'

In [51]:
parse_one_json_article('C:/Users/oae15/Downloads/Compressed/out2019json/AA/wiki_00', 'C:/Users/oae15/Downloads/Compressed/out2019json/AA/wiki_00_one.txt')

DONE :)


In [78]:
%%time
path = Path(r'C:\Users\oae15\Downloads\Compressed\out2019json\AA') # path of WikiExtractor.py output
articles = parse_json_articles(path)
print('# articles:', len(articles))

# articles: 600559
Wall time: 10.7 s


In [3]:
def how_many_words(all_articles):
    vocab = []
    for article in all_articles:
        article_words = article['text'].split()
        vocab.extend(article_words)
    unique_vocab = set(vocab)
    return len(vocab), len(unique_vocab)

In [80]:
%%time
n_words, n_uniq_words = how_many_words(articles)
print('# words: {}, # unique words: {}'.format(n_words,n_uniq_words))

# words: 108658427, # unique words: 4166467
Wall time: 42.1 s


__Note fron Jeremy Howard:__
`A note to those folks building langage models: there’s no reason to go beyond 100 million tokens - in my experiments it didn’t help. So if your corpus is bigger than that, remove some of the smaller articles (for instance) until the corpus is down to that size. Really large corpuses are a pain to work with, and don’t have any benefits I saw.`

We got 108M words, so I guess we are good to go :)

In [91]:
%%time
preprocess_articles_and_write_to_file(articles, r'C:\Users\oae15\Downloads\Compressed\out2019json\all_arabic_wiki_2019.txt')

100%|████████████████████████████████████████████████████████████████████████| 600559/600559 [01:24<00:00, 7070.72it/s]


DONE :)
Wall time: 1min 24s


__Now, all articles are in all_arabic_wiki_2019.txt. Each line is an article. It's time to start preprocessing using MADAMIRA!__

In [6]:
print(articles[0]['text'])

ماء

الماء مادةٌ شفافةٌ عديمة اللون والرائحة، وهو المكوّن الأساسي للجداول والبحيرات والبحار والمحيطات وكذلك للسوائل في جميع الكائنات الحيّة، وهو أكثر المركّبات الكيميائيّة انتشاراً على سطح الأرض. يتألّف جزيء الماء من ذرّة أكسجين مركزية ترتبط بها ذرّتي هيدروجين برابطة تساهميّة لتكون صيغته HO. عند الظروف القياسية من الضغط ودرجة الحرارة يكون الماء سائلاً، ولكنّ حالاته الأخرى شائعة الوجود أيضاً؛ وهي حالة الجليد الصلبة والبخار الغازيّة.

إنّ الماء هو أساس وجود الحياة على كوكب الأرض، وهو يغطّي 71% من سطحها، وتمثّل مياه البحار والمحيطات أكبر نسبة للماء على الأرض، حيث تبلغ حوالي 96.5%. وتتوزّع النسب الباقية بين المياه الجوفيّة وبين جليد المناطق القطبيّة (1.7% لكليهما)، مع وجود نسبة صغيرة على شكل بخار ماء معلّق في الهواء على هيئة سحاب (غيوم)، وأحياناً أخرى على هيئة ضباب أو ندى، بالإضافة إلى الزخات المطريّة أو الثلجيّة. تبلغ نسبة الماء العذب حوالي 2.5% فقط من الماء الموجود على الأرض، وأغلب هذه الكمّيّة (حوالي 99%) موجودة في الكتل الجليديّة في المناطق القطبيّة، في حين تتواجد 0.3% من الماء العذب ف

### After MADAMIRA

In [4]:
%%time
mada_articles = []
path = Path(r'C:\Python\Python36\coding4fun\Notebooks\fastai\for research\Ar_LM\data') # path of WikiExtractor.py output
with open(path / 'all_arabic_wiki_2019.txt.MyD3.tok', encoding='utf-8') as file: # open madamira output file
    for line in file.readlines(): # loop through file lines. Each line is an article formatted in json
        line = line.strip() # remove leading and trailing whitespaces (and new lines)
        mada_articles.append(line)
print('# articles:', len(mada_articles))

# articles: 600559
Wall time: 6.14 s


In [5]:
%%time
vocab = []
for article in mada_articles:
    article_words = article.split()
    vocab.extend(article_words)
unique_vocab = set(vocab)
print('# words: {}, # unique words: {}'.format(len(vocab),len(unique_vocab)))

# words: 157522583, # unique words: 914414
Wall time: 25.6 s


In [6]:
mada_articles[0]

'ماء  ال+ ماء  ماده  شفافه  عديمه  ال+ لون  و+ ال+ راءحه  ,  و+ هو  ال+ مكون  ال+ اساسي  ل+ ال+ جداول  و+ ال+ بحيرات  و+ ال+ بحار  و+ ال+ محيطات  و+ ك+ ذلك  ل+ ال+ سواءل  في  جميع  ال+ كاءنات  ال+ حيه  ,  و+ هو  اكثر  ال+ مركبات  ال+ كيمياءيه  انتشارا  علي  سطح  ال+ ارض  .  يتالف  جزيء  ال+ ماء  من  ذره  اكسجين  مركزيه  ترتبط  ب +ها ذرتي  هيدروجين  ب+ رابطه  تساهمي +ه ل+ تكون  صيغه +ه .  عند  ال+ ظروف  ال+ قياسيه  من  ال+ ضغط  و+ درجه  ال+ حراره  يكون  ال+ ماء  ساءلا  ,  و+ لكن  حالات +ه ال+ اخري  شاءعه  ال+ وجود  ايضا  ,  و+ هي  حاله  ال+ جليد  ال+ صلبه  و+ ال+ بخار  ال+ غازيه  .  ان  ال+ ماء  هو  اساس  وجود  ال+ حياه  علي  كوكب  ال+ ارض  ,  و+ هو  يغطي  رقم  من  سطح +ها ,  و+ تمثل  مياه  ال+ بحار  و+ ال+ محيطات  اكبر  نسبه  ل+ ال+ ماء  علي  ال+ ارض  ,  حيث  تبلغ  حوال +ي رقم  .  و+ تتوزع  ال+ نسب  ال+ باقيه  بين  ال+ مياه  ال+ جوفيه  و+ بين  جليد  ال+ مناطق  ال+ قطبيه  ,  مع  وجود  نسبه  صغيره  علي  شكل  بخار  ماء  معلق  في  ال+ هواء  علي  هيءه  سحاب  ,  و+ احيانا  اخري  علي  هيءه  ض